# Sous problème - Bellman-Ford

Dans ce notebook, nous définissons la seconde méthode de résolution du sous problème de la génération de colonnes.

In [ ]:
function sub_pb_BF(A,W,p,n,K,preds,eps)
    """
    Résout le sous problème de la génération de colonnes avec l'algorithme de Bellman-Ford

    Entrées :
        - A     : Array   - Matrice des arcs du graphe
        - W     : Array   - Matrice des poids de chaque arc
        - p     : Array   - Solution optimale du dual de la relaxation
        - n     : Integer - Nombre de sommets dans le graphe
        - K     : Integer - Taille maximale des cycles
        - preds : Array   - Liste des prédecesseurs de chaque sommet
        - eps   : Float   - Tolérance pour la comparaison à 0
    
    Sorties :   
        - cycle      : Array - Cycle de coût positif identifié
        - cout_cycle : Float - Coût de ce cycle
    """
    # On démarre de chaque noeud origine possible
    for o in 1:(n-1)
        #############################################
        ### Recherche des meilleurs prédécesseurs ###
        #############################################
        # On va chercher les chemins de coût max allant aux autres sommets, en au plus K arcs
        # Historique des distances / prédécesseurs
        Dist = []
        Pred = []
        
        # Distances courantes pour aller de o à chaque sommet
        distances = [-Inf for i in 1:n]
        distances[o] = 0
        push!(Dist,copy(distances))
        
        # Liste courante des meilleurs prédecesseur de chaque noeud
        predecessors = [-1 for i in 1:n]
        push!(Pred,copy(predecessors))
        
        # Pour des chemins de longueur <= K
        for k in 1:K  
            # On parcourt tous les arcs du graphe
            for a in A
                # Et on met à jour les distances max entre o et chaque noeud (en mettant à jour leur prédécesseur)
                if Dist[k][a[1]]>-Inf && Dist[k][a[1]] + (W[a[1],a[2]] - p[a[2]]) > Dist[k][a[2]]
                    distances[a[2]] = Dist[k][a[1]] + (W[a[1],a[2]] - p[a[2]])
                    predecessors[a[2]] = a[1]
                end
            end 
            # On ajoute les distances / prédécesseurs courants dans les listes historiques
            push!(Dist,copy(distances))
            push!(Pred,copy(predecessors))
        end

        #############################################
        ###   Meilleur prédécesseur de l'origine  ###
        #############################################
        # Meilleur prédécesseur de l'origine
        best_pred = Pred[K+1][o]
        
        # Si l'origine a un prédécesseur
        if best_pred != -1
            #############################################
            ###   Identification du meilleur cycle    ###
            #############################################
            # Initialisation du coût du cycle
            cout_cycle = 0
            # On ajoute le premier arc du cycle, en stockant le prédécesseur ajouté, et en mettant à jour le coût
            cycle = [(best_pred,o)]
            pred_cycle = [best_pred]
            cout_cycle += W[best_pred,o] - p[o]
            
            # On remonte les arcs pour construire le cycle
            cpt = K
            # Sommet d'arrivée du dernier arc ajouté
            last = o
            # Booléen pour la gestion des sous-cycles
            test_subcycle = false
            
            # Tant qu'un cycle n'est pas obtenu ou qu'un sous-cycle n'a pas été identifié
            while best_pred != o && !test_subcycle
                # On cherche la dernière mise à jour du prédécesseur de "last" (sommet d'arrivée du dernier arc ajouté)
                if (Pred[cpt][last] == best_pred)
                    cpt -= 1
                else 
                    # Identification des deux sommets de l'arc suivant
                    temp = best_pred
                    best_pred = Pred[cpt][temp]

                    # Si le prédécesseur de l'arc n'a pas encore été visité, on ajoute l'arc
                    if !(best_pred in pred_cycle) 
                        # Ajout de l'arc dans le cycle
                        push!(cycle,(best_pred,temp))
                        # Mise à jour du coût
                        cout_cycle += W[best_pred,temp] - p[temp]
                        # Mise à jour de l'historique des prédécesseurs
                        push!(pred_cycle,best_pred)
                        # On décrémente le compteur et on met à jour "last"
                        cpt -= 1
                        last = temp
                    else
                        # Sinon, il existe un sous-cycle
                        # Ajout du premier arc du sous-cycle
                        subcycle = [(best_pred,temp)]
                        # Initialisation de son coût
                        cout_subcycle = W[best_pred,temp] - p[temp]
                        # Sommet "pivot", ayant 2 prédécesseurs
                        pivot = temp
                        # Tant qu'un cycle n'est pas obtenu
                        while pivot != best_pred
                            # On ajoute le dernier arc du cycle construit jusqu'alors
                            arc = pop!(cycle)
                            push!(subcycle,arc)
                            # Et on met à jour le pivot, ainsi que le coût du sous-cycle
                            pivot = arc[2]
                            cout_subcycle += W[arc[1],arc[2]] - p[arc[2]]
                        end
                        # Opération annulant le reverse de sortie
                        cycle = reverse(subcycle)
                        # Mise à jour du cout du cycle
                        cout_cycle = cout_subcycle
                        # On actualise la booléen pour sortir de la boucle
                        test_subcycle = true
                    end
                end
            end
            # On remet le cycle dans le bon ordre 
            cycle = reverse(cycle)
                
            #############################################
            ##  Décision en fonction du coût du cycle  ##
            #############################################
            # On renvoie le cycle et son coût si celui-ci est positif
            if cout_cycle > eps
                return cycle, cout_cycle
            end
            
        end
        # Si on arrive ici, cela signifie qu'il n'y a pas de cycle positif passant par o
        # --> On supprime les arcs contenant o dans l'itération suivante
        A = filter(arc -> !(o in arc), A)
    end
    # Si on arrive ici, c'est que le cycle de coût max est négatif 
    # --> on ne renvoie rien
    return nothing, 0
end

In [1]:
println("Résolution par Bellman-Ford importée avec succès.")

Résolution par Bellman-Ford importée avec succès.
